# Transient 1D Wave Simulation

This notebook describes how to set up a transient simulation with partial updates. This is illustrated with a simple simulation of a wave propagating with a velocity of 1m/s and a damping coefficient of 0.5 1/s. The domain is 2m long with the end at x = 0m fixed and the end at x = 2m free.

The wave equation with damping:

$$
v^2 \frac{\partial^2 u}{\partial x^2} - \frac{\partial^2 u}{\partial t^2} - b \frac{\partial u}{\partial t} = 0 
$$

This will be solved with 4th order accuracy for the second derivatives and fifth order for the damping term so that the number of coefficients match in the approximation of the time derivative. The domain (x) is discretized by 5 terms while time is discretized with a backwards approximation with 6 terms.

Here's the stencil:
$$
\begin{matrix}
u_{x-2}^t & u_{x-1}^t & u_{x}^t & u_{x+1}^t & u_{x+2}^t\\
\hline
&&u_x^{t-1}\\
&&u_x^{t-2}\\
&&u_x^{t-3}\\
&&u_x^{t-4}\\
&&u_x^{t-5}\\
\end{matrix}
$$

Terms above the line in the stencil are solved for each time step. The terms below the line are static. As a result, the terms above the line end up in the model coefficient matrix while the static coefficients below the line are put in the constraint vector.

The wave equation is re-written to reflect this. Note the sign change when moving them across to the RHS.

$$
v^2 \frac{\partial^2 u^t}{\partial x^2} - \frac{\partial^2 u^t}{\partial t^2} - b \frac{\partial u^t}{\partial t} = \frac{\partial^2 u^{t-n}}{\partial t^2} + b \frac{\partial u^{t-n}}{\partial t}
$$

This is easily accomplished in FastFD using `Scalar.dt('lhs')` and `Scalar.dt('rhs')`

When solving a transient simulation, the constraint vector needs to be updated during each iteration by multiplying `Scalar.dt('rhs')` by the unraveled time history stacked along the a new last axis. In 1 dimension for this example, this looks like:

$$
\text{Scalar.dt('rhs')} \times \left(\begin{matrix}
u_{x=0}^{t-5} & u_{x=0}^{t-4} & u_{x=0}^{t-3} & u_{x=0}^{t-2} & u_{x=0}^{t-1}\\
u_{x=1}^{t-5} & u_{x=1}^{t-4} & u_{x=1}^{t-3} & u_{x=1}^{t-2} & u_{x=1}^{t-1}\\
u_{x=2}^{t-5} & u_{x=2}^{t-4} & u_{x=2}^{t-3} & u_{x=2}^{t-2} & u_{x=2}^{t-1}\\
\vdots\\
u_{x=n}^{t-5} & u_{x=n}^{t-4} & u_{x=n}^{t-3} & u_{x=n}^{t-2} & u_{x=n}^{t-1}\\
\end{matrix}\right) \text{.ravel()}
$$

Another thing to note is that for this problem, only the constraint vector needs to be updated inside the iterative loop. This can be done by calling the following:
```python
FDModel.update_equations({'key': (None, new_vector)})
```

In [ ]:
import fastfd as ffd
ffd.sparse_lib('scipy')

import numpy as np

In [ ]:
acc = 4 # Global accuracy order

x = ffd.LinearAxis('x', start = 0, stop = 2, num = 501)
u = ffd.Scalar('u', [x], accuracy = acc)
model = ffd.FDModel([u], timestep = 0.0005) # timestep is set globally for the model

In [ ]:
v = 1 # acoustic velocity (m/s)
b = 0.5 # damping coefficient (1/s)

model.update_equations({
    'u': (v**2 * u.d('x', derivative = 2) \
          - u.dt('lhs', derivative = 2) \
          - b * u.dt('lhs', accuracy = acc + 1), # Damping term has +1 accuracy so the shapes match
          None), # The constraint vector is set for each iteration so can be left empty here.
})

model.update_bocos({
    'fixed x0': (u.i[0], u.i[0], 0), # amplitude at x = 0 is set to zero
    'free x1': (u.i[-1], u.d('x')[-1], 0), # the end at x = 2 is free, so du/dx = 0.
})

 # pre-compute the RHS coefficient matrix outside the iterative loop to reduce overheads
dt_const = u.dt('rhs', derivative = 2) + b * u.dt('rhs', accuracy = acc + 1)

def update_timestep(u0):
    # Update the constraint vector inside the iterative loop
    model.update_equations({
        'u': (None, dt_const * u0.ravel()),
    })
    
    return model.solve()

In [ ]:
# Set initial conditions. This could be nearly any shape you choose
def init(x_, t):
    coord = (.75 - x_ * x.delta + 0 * v * t * model.timestep) / 0.1 # delete '0 *' to start with a moving wave
    return 0.1 * (coord**2 + 1)**(-3/2) * np.cos(2 * np.pi * coord / 1.5) 

u0 = np.fromfunction(init, (x.num_points, acc + 1))

#Solve the problem to 16001 timesteps. A large number of very small timesteps are required to avoid instability.
result = []
for i in range(16001):
    res = update_timestep(u0)['u']
    if i%160==0: # save a result every 0.08s
        result.append(res)
        
    # update the time history. drop the first column and append the new result at the end along axis 1.
    u0 = np.column_stack([u0[:,1:], res])

In [ ]:
# plot the results!
import holoviews as hv
hv.extension('bokeh')
hv.output(widget_location='bottom')

result = hv.HoloMap({
    t * model.timestep * 160: hv.Curve((model.coords['u']['x'], res))
    for t, res in enumerate(result)
}, kdims = 'Time (s)').opts(
    width = 700, height = 500,
    tools = ['hover'], show_grid = True,
    xlabel = 'x (m)', ylabel = 'Amplitude (m)'
)

In [ ]:
hv.save(result, 'transient_wave.html')